In [107]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Gathering + Filtering Data

In [108]:
# INPUT DATA
df = pd.read_csv(r"..\InputData\RidershipData.csv")
df.head()

,Route,Day,Ride Start,Stop,Scheduled Time,Day Of Week,Vehicle Capacity,Ride State,Stop State,Actual Arrival,Actual Departure,Riders On,Riders Off,Riders Left,Riders Cumulative
0,Campus Shuttle,2023/08/31,2023-08-31 07:30:00,South St and Turner Rd,2023-08-31 07:34:00,Thursday,45.0,Complete,Departed,2023-08-31 07:54:39,2023-08-31 07:54:52,0,0,0,0
1,Campus Shuttle,2023/08/31,2023-08-31 07:30:00,Charles River parking lot,2023-08-31 07:35:00,Thursday,45.0,Complete,Departed,2023-08-31 07:55:27,2023-08-31 07:59:10,0,0,0,0
2,Campus Shuttle,2023/08/31,2023-08-31 07:30:00,Charles River Apts,2023-08-31 07:37:00,Thursday,45.0,Complete,Departed,2023-08-31 08:01:02,2023-08-31 08:01:23,0,0,0,0
3,Campus Shuttle,2023/08/31,2023-08-31 07:30:00,Charles River parking lot,2023-08-31 07:38:00,Thursday,45.0,Complete,Departed,2023-08-31 08:02:24,2023-08-31 08:02:34,0,0,0,0
4,Campus Shuttle,2023/08/31,2023-08-31 07:30:00,South St and Turner Rd,2023-08-31 07:54:00,Thursday,45.0,Complete,Departed,2023-08-31 08:03:07,2023-08-31 08:04:03,0,0,0,0


In [109]:
# Convert 'Scheduled Time' to datetime and create half-hour increments
df['Scheduled Time'] = pd.to_datetime(df['Scheduled Time'])
df['Time Slot'] = df['Scheduled Time'].dt.floor('30T').dt.strftime('%I:%M %p')

# Create a helper column for sorting
df['Time Slot Sort'] = df['Scheduled Time'].dt.floor('30T').dt.time

# Create a list of unique days of the week
days_of_week = df['Day Of Week'].unique()

# Create a time range in ten-minute increments
time_range = pd.date_range(start="00:00", end="23:59", freq='10T').time

# Create tick values and labels for the slider (show every hour)
tick_vals = [i for i in range(len(time_range)) if time_range[i].minute == 0]
tick_labels = [time_range[i].strftime('%I:%M %p') for i in tick_vals]

# Creating Plotly Dashboard

In [110]:
# Initialize the Dash app
app = dash.Dash(__name__)

In [111]:
app.layout = html.Div([
    html.H1("Riders Analysis Dashboard"),
    
    # Route Dropdown
    html.Label("Select Route:"),
    dcc.Dropdown(
        id='route-dropdown',
        options=[{'label': route, 'value': route} for route in df['Route'].unique()],
        value=df['Route'].unique()[0]
    ),
    
    # Sum/Average Dropdown
    html.Label("Select Metric:"),
    dcc.Dropdown(
        id='metric-dropdown',
        options=[
            {'label': 'Sum', 'value': 'Sum'},
            {'label': 'Average', 'value': 'Average'}
        ],
        value='Average'
    ),

    # Date Range Slider
    html.Label("Select Date Range:"),
    dcc.DatePickerRange(
        id='date-range-picker',
        min_date_allowed=df['Scheduled Time'].min().date(),
        max_date_allowed=df['Scheduled Time'].max().date(),
        initial_visible_month=df['Scheduled Time'].min().date(),
        start_date=df['Scheduled Time'].min().date(),
        end_date=df['Scheduled Time'].max().date()
    ),

    # Day of Week Multi-Select
    html.Label("Select Day(s) of Week:"),
    dcc.Dropdown(
        id='day-of-week-dropdown',
        options=[{'label': day, 'value': day} for day in days_of_week],
        value=days_of_week,
        multi=True
    ),
    
    # Time Range Slider
    html.Label("Select Time Range:"),
    dcc.RangeSlider(
        id='time-range-slider',
        min=0,
        max=len(time_range)-1,
        value=[0, len(time_range)-1],
        marks={i: label for i, label in zip(tick_vals, tick_labels)},
        step=1,
        allowCross=False
    ),
    
    # Graph
    dcc.Graph(id='riders-bar-chart')
])

In [112]:
@app.callback(
    Output('riders-bar-chart', 'figure'),
    [Input('route-dropdown', 'value'),
     Input('metric-dropdown', 'value'),
     Input('time-range-slider', 'value')]
)
def update_bar_chart(selected_route, selected_metric, selected_time_range):
    # Filter data by selected route
    filtered_df = df[df['Route'] == selected_route]

    # Filter by selected time range
    start_time = time_range[selected_time_range[0]]
    end_time = time_range[selected_time_range[1]]
    filtered_df = filtered_df[(filtered_df['Time Slot Sort'] >= start_time) & (filtered_df['Time Slot Sort'] <= end_time)]

    # Group by time slot and calculate sum or average
    if selected_metric == 'Sum':
        grouped_df = filtered_df.groupby(['Time Slot', 'Time Slot Sort']).sum(numeric_only=True)[['Riders On', 'Riders Off']]
    else:
        grouped_df = filtered_df.groupby(['Time Slot', 'Time Slot Sort']).mean(numeric_only=True)[['Riders On', 'Riders Off']]
    
    grouped_df.reset_index(inplace=True)
    
    # Sort by the helper column
    grouped_df.sort_values('Time Slot Sort', inplace=True)
    
    # Create the stacked bar chart
    fig = px.bar(grouped_df, x='Time Slot', y=['Riders On', 'Riders Off'],
                 labels={'value': 'Number of Riders', 'Time Slot': 'Time'},
                 title=f"{selected_metric} of Riders On and Riders Off for Route {selected_route}",
                 barmode='stack')

    fig.update_layout(xaxis={'categoryorder': 'array', 'categoryarray': grouped_df['Time Slot']})

    return fig

SyntaxError: incomplete input (231215805.py, line 26)

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=True, port=1002)